In [1]:
import sys
import os

import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import auc, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from scipy.spatial.distance import pdist, squareform

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['font.sans-serif'] = ['FangSong']
matplotlib.rcParams['axes.unicode_minus'] = False 
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import warnings
warnings.filterwarnings('ignore')

data_origin = pd.read_csv('C:/Users/Ca1/Desktop/papers/Nocturnal_hypertension by C/data_continuous_gbk_20201211.csv', encoding = 'gbk', index_col = 0)
#data_cut = pd.read_csv('datasets/data_categorical_gbk_1117.csv', encoding = 'gbk', index_col = 0)
data_range = data_origin[['Male','Age,y','BMI,kg/m2','Smoker','Alcohol intake','Diabetes mellitus','Hypertension','Hyperlipidemia'
,'Hypertension family history','CVD history','Antihypertension drugs','RAAS drugs intake','nRAAS drugs intake'
,'Proteinuria,mg/24h','Hemoglobin,g/L','BUN,mmol/L','Scr,mmol/L','Uric acid,mmol/L','FBG,mmol/L','Triglyceride,mmol/L'
,'Cholesterol,mmol/L','HDL-C,mmol/L','LDL-C,mmol/L','Serum albumin,g/L','Serum sodium,mmol/L','Serum calcium,mmol/L'
,'Serum phosphate,mmol/L','β2-MG,ug/mL','iPTH,pg/mL','Homocysteine,umol/L','eGFR,mL/ (min·1.73 m2)'
,'Clinic SBP,mmHg','Clinic DBP,mmHg','nocturnal_hypertension']]

def getXYcol(data0):
    X_col = [col for col in data0.columns if col != 'nocturnal_hypertension']
    return X_col, 'nocturnal_hypertension'

def preprocess(data0):
    data = data0.copy()
    X_col = [col for col in data.columns if col != 'nocturnal_hypertension']
    dummy_col = [col for col in X_col if len(data[col].unique()) < 5]
    data[dummy_col] = data_range[dummy_col].astype('int64')
    float_col = [col for col in X_col if col not in dummy_col]
    # standard
    m,s = data[float_col].mean(), data[float_col].std()
    for col in float_col:
        data[col] = (data[col] - data[col].mean()) / (data[col].std())
    temp = pd.DataFrame()
    for col in dummy_col:
        temp = pd.concat([temp, pd.get_dummies(data[col], prefix = col, prefix_sep = '_', drop_first = True)], axis = 1)
    temp = pd.concat([data[float_col], temp, data['nocturnal_hypertension']], axis = 1)
    temp.index = pd.Index(range(len(temp)))
    X_col, y_col = getXYcol(temp)
    
    return temp, m, s, float_col, dummy_col, X_col, y_col

#data_cut_pro, MEAN_CUT, STD_CUT = preprocess(data_cut)
data_range_pro, m,s, float_col, dummy_col, X_col, y_col = preprocess(data_range)

#data_range_pro.to_csv('C:/Users/coldog/Desktop/temp.csv', encoding= 'gbk', index ='False')

# =============================================================================
# model.fit
# =============================================================================
def model(data0, classifier):
    X_col, Y_col = getXYcol(data0)
    train_auc = []
    test_auc = []
    train_acc = []
    test_acc = []
    
    train_spe = []
    test_spe = []
    train_sen = []
    test_sen = []
    train_F1 = []
    test_F1 = []
    kf = KFold(10,shuffle = True, random_state = 115306)
    for train_idx, test_idx in kf.split(range(len(data0))):
        X_train = data0[X_col].iloc[train_idx]
        y_train = data0[y_col].iloc[train_idx].astype('uint8')
        X_test = data0[X_col].iloc[test_idx]
        y_test = data0[y_col].iloc[test_idx].astype('uint8')
        
        
        logit = classifier
        
        sw = y_train.copy()
        sw[sw == 1] = y_train.shape[0]/(2*y_train.sum())
        sw[sw == 0] = y_train.shape[0]/(2*(y_train==0).sum())
        
        result = logit.fit(X_train,y_train.values.reshape(len(y_train)), sample_weight = sw)
        
        # auc
        train_pred_proba = result.predict_proba(X_train)
        test_pred_proba = result.predict_proba(X_test)
        train_fpr, train_tpr, train_thresholds = roc_curve(y_train, train_pred_proba[:,1])
        test_fpr, test_tpr, test_thresholds = roc_curve(y_test, test_pred_proba[:,1])
        
        train_auc.append(auc(train_fpr, train_tpr))
        test_auc.append(auc(test_fpr, test_tpr))
        
        # acc
        train_pred = result.predict(X_train)
        test_pred = result.predict(X_test)
        train_acc.append((train_pred == y_train.values.reshape(-1)).sum() / len(train_pred))
        test_acc.append((test_pred == y_test.values.reshape(-1)).sum() / len(test_pred))
        
        # specificity
        train_pred = result.predict(X_train.loc[y_train==0])
        test_pred = result.predict(X_test.loc[y_test==0])
        train_spe.append((train_pred == 0).sum() / len(train_pred))
        test_spe.append((test_pred == 0).sum() / len(test_pred))
        
        # sensitivity
        train_pred = result.predict(X_train.loc[y_train==1])
        test_pred = result.predict(X_test.loc[y_test==1])
        train_sen.append((train_pred == 1).sum() / len(train_pred))
        test_sen.append((test_pred == 1).sum() / len(test_pred))
        
        # F1
        train_pred = result.predict(X_train)
        test_pred = result.predict(X_test)
        train_pre = ((train_pred==1)&(y_train == 1)).sum()/(train_pred == 1).sum()
        test_pre = ((test_pred==1)&(y_test == 1)).sum()/(test_pred == 1).sum()
        train_F1.append((2*train_pre*train_sen[-1])/(train_pre + train_sen[-1]))
        test_F1.append((2*test_pre*test_sen[-1])/(test_pre + test_sen[-1]))
        

        
    return (np.mean(train_auc), np.mean(test_auc), np.mean(train_acc), np.mean(test_acc), 
            np.mean(train_spe), np.mean(test_spe), np.mean(train_sen), np.mean(test_sen),
            np.mean(train_F1), np.mean(test_F1))


In [2]:
# =============================================================================
# single
# =============================================================================
X_col, Y_col = getXYcol(data_range_pro)
result = {}
for col in X_col:
#     print(col)
    result[col] = model(data_range_pro[[col, Y_col]], ensemble.RandomForestClassifier(
        n_estimators = 100,min_samples_leaf = 50, class_weight = 'balanced', oob_score = True))[1]
#     print(result[col])

In [3]:
k,v = np.array(list(result.keys())), np.array(list(result.values()))
i = np.argsort(v)
# plt.figure(figsize=(20,16))
# plt.title('Single Feature Valid-AUC in RandomForest',fontsize=30)
# plt.barh(range(len(k)),v[i],color='lightblue',align='center')
# plt.yticks(range(len(k)),k[i],fontsize=18)
# plt.ylim([-1,len(k)])
# plt.tight_layout()
# plt.show()

In [4]:
# =============================================================================
# CV multi
# =============================================================================
result_cv = {}
for j in range(len(i)):
    cols = k[i[::-1][:(j+1)]]
    result_cv[j] = model(data_range_pro[list(cols)+[Y_col]], ensemble.RandomForestClassifier(
        n_estimators = 100,min_samples_leaf = 10, class_weight = 'balanced', oob_score = True))
#     print(j, 'done')

In [7]:
dataday = pd.read_csv('C:/Users/Ca1/Desktop/papers/Nocturnal_hypertension by C//dataday_20201211.csv',encoding = 'gbk', index_col = 0)
data0 = data_origin
data = pd.merge(data0,dataday,on = 'No.',how = 'left')
data = data[['Clinic SBP,mmHg','eGFR,mL/ (min·1.73 m2)','BUN,mmol/L','Clinic DBP,mmHg'
             ,'β2-MG,ug/mL','nRAAS drugs intake', 'Hypertension','Scr,mmol/L'
             ,'Age,y','Homocysteine,umol/L'
             ,'10-11DBP', '10-11SBP', '10-12DBP', '10-12SBP', '11-12DBP', '11-12SBP'
             ,'3-4DBP', '3-4SBP', '3-5DBP', '3-5SBP', '4-5DBP', '4-5SBP', '4-6DBP','4-6SBP'
             , '5-6DBP', '5-6SBP', '8-10DBP', '8-10SBP'
             , '8-9DBP', '8-9SBP', '9-10DBP', '9-10SBP', '9-11DBP','9-11SBP','nocturnal_hypertension']]

In [9]:
data.to_csv('C:/Users/Ca1/Desktop/papers/Nocturnal_hypertension by C/data_10factors_gbk_dayBP_201222.csv', encoding= 'gbk', index ='False')